<a href="https://colab.research.google.com/github/MoizAhmed2517/Deep_Learning_Projects/blob/main/Yolo_Multi_object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi Object Detection Yolo

In [1]:
import os
from glob import glob
import pandas as pd
from functools import reduce
from xml.etree import ElementTree as et
import zipfile 
from google.colab import drive

In [2]:
!wget https://drive.google.com/file/d/1gJEhVWgUMbYq6X3YjI_2PveQiYKCSUy0/view?usp=share_link

--2022-12-06 18:46:35--  https://drive.google.com/file/d/1gJEhVWgUMbYq6X3YjI_2PveQiYKCSUy0/view?usp=share_link
Resolving drive.google.com (drive.google.com)... 142.251.162.101, 142.251.162.102, 142.251.162.113, ...
Connecting to drive.google.com (drive.google.com)|142.251.162.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘view?usp=share_link’

view?usp=share_link     [ <=>                ]  72.88K  --.-KB/s    in 0.002s  

2022-12-06 18:46:35 (30.3 MB/s) - ‘view?usp=share_link’ saved [74630]



In [3]:
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Datasets/data-images.zip")
zip_ref.extractall()
zip_ref.close()

In [4]:
xml_lst = glob("/content/data_images/*.xml")